In [4]:
# Required libraries
import os
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, Subset

# Set device (GPU/CPU)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Load dataset
data_dir = '/kaggle/input/defungi'  # Change this to your dataset path

# Define basic transformations
transform = transforms.Compose([
    transforms.Resize((64,64)),
    transforms.ToTensor(),
])

# Load full dataset
full_dataset = datasets.ImageFolder(root=data_dir, transform=transform)

# Split data into labeled (small fraction) and unlabeled
train_indices, test_indices = train_test_split(list(range(len(full_dataset))), test_size=0.2, stratify=full_dataset.targets)
labeled_indices, unlabeled_indices = train_test_split(train_indices, test_size=0.9, stratify=np.array(full_dataset.targets)[train_indices])

labeled_dataset = Subset(full_dataset, labeled_indices)
unlabeled_dataset = Subset(full_dataset, unlabeled_indices)
test_dataset = Subset(full_dataset, test_indices)

# DataLoaders
batch_size = 32
labeled_loader = DataLoader(labeled_dataset, batch_size=batch_size, shuffle=True)
unlabeled_loader = DataLoader(unlabeled_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

# Define CNN model
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.pool = nn.MaxPool2d(2, 2)
        self.fc1 = nn.Linear(64 * 16 * 16, 128)
        self.fc2 = nn.Linear(128, 6)  # 6 fungi classes

    def forward(self, x):
        x = self.pool(nn.ReLU()(self.conv1(x)))
        x = self.pool(nn.ReLU()(self.conv2(x)))
        x = x.view(-1, 64 * 16 * 16)
        x = nn.ReLU()(self.fc1(x))
        x = self.fc2(x)
        return x

# Define training and evaluation function
def train_model(model, optimizer, criterion, labeled_loader, unlabeled_loader, epochs=5):
    model.train()
    for epoch in range(epochs):
        total_loss = 0
        for (labeled_images, labeled_targets), (unlabeled_images, _) in zip(labeled_loader, unlabeled_loader):
            labeled_images, labeled_targets = labeled_images.to(device), labeled_targets.to(device)
            unlabeled_images = unlabeled_images.to(device)

            optimizer.zero_grad()

            # Forward pass on labeled data
            labeled_outputs = model(labeled_images)
            labeled_loss = criterion(labeled_outputs, labeled_targets)

            # Forward pass on unlabeled data
            pseudo_labels = model(unlabeled_images).detach().argmax(dim=1)
            unlabeled_outputs = model(unlabeled_images)
            unlabeled_loss = criterion(unlabeled_outputs, pseudo_labels)

            # Combined loss
            loss = labeled_loss + unlabeled_loss
            loss.backward()
            optimizer.step()

            total_loss += loss.item()

        print(f'Epoch [{epoch+1}/{epochs}], Loss: {total_loss / len(labeled_loader):.4f}')

# FixMatch model (consistency regularization)
class FixMatch(nn.Module):
    def __init__(self, model, tau=0.95):
        super(FixMatch, self).__init__()
        self.model = model
        self.tau = tau

    def forward(self, x_weak, x_strong):
        logits_weak = self.model(x_weak)
        pseudo_labels = torch.softmax(logits_weak, dim=1).argmax(dim=1)
        mask = torch.max(torch.softmax(logits_weak, dim=1), dim=1)[0] >= self.tau
        logits_strong = self.model(x_strong)
        return logits_strong, pseudo_labels, mask

# Mean Teacher model
class MeanTeacher(nn.Module):
    def __init__(self, student, teacher, alpha=0.99):
        super(MeanTeacher, self).__init__()
        self.student = student
        self.teacher = teacher
        self.alpha = alpha

    def update_teacher(self):
        for teacher_param, student_param in zip(self.teacher.parameters(), self.student.parameters()):
            teacher_param.data = self.alpha * teacher_param.data + (1 - self.alpha) * student_param.data

    def forward(self, x):
        return self.student(x), self.teacher(x)

# MixMatch augmentation
def mixmatch(x_labeled, y_labeled, x_unlabeled, model, T=0.5, K=2):
    with torch.no_grad():
        preds_unlabeled = [model(x_unlabeled) for _ in range(K)]
        preds_unlabeled = torch.mean(torch.stack(preds_unlabeled), dim=0)
        pseudo_labels = torch.softmax(preds_unlabeled / T, dim=1)
    all_inputs = torch.cat([x_labeled, x_unlabeled], dim=0)
    all_labels = torch.cat([y_labeled, pseudo_labels.argmax(dim=1)], dim=0)
    lam = np.random.beta(0.75, 0.75)
    index = torch.randperm(all_inputs.size(0))
    mixed_inputs = lam * all_inputs + (1 - lam) * all_inputs[index]
    mixed_labels = lam * all_labels + (1 - lam) * all_labels[index]
    return mixed_inputs, mixed_labels

# Model evaluation
def evaluate_model(model, test_loader):
    model.eval()
    all_preds = []
    all_labels = []
    with torch.no_grad():
        for images, labels in test_loader:
            images = images.to(device)
            outputs = model(images)
            preds = torch.argmax(outputs, dim=1)
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.numpy())
    print(classification_report(all_labels, all_preds, target_names=full_dataset.classes))

# Train and evaluate FixMatch model
cnn_model = CNN().to(device)
fixmatch_model = FixMatch(cnn_model).to(device)
optimizer = optim.Adam(fixmatch_model.parameters(), lr=0.001)
criterion = nn.CrossEntropyLoss()

train_model(fixmatch_model.model, optimizer, criterion, labeled_loader, unlabeled_loader, epochs=10)
print("FixMatch Model Evaluation:")
evaluate_model(fixmatch_model.model, test_loader)

# Train and evaluate Mean Teacher model
student_model = CNN().to(device)
teacher_model = CNN().to(device)
mean_teacher_model = MeanTeacher(student_model, teacher_model).to(device)

train_model(mean_teacher_model.student, optimizer, criterion, labeled_loader, unlabeled_loader, epochs=10)
print("Mean Teacher Model Evaluation:")
evaluate_model(mean_teacher_model.student, test_loader)

# Train and evaluate MixMatch model
train_model(cnn_model, optimizer, criterion, labeled_loader, unlabeled_loader, epochs=10)
print("MixMatch Model Evaluation:")
evaluate_model(cnn_model, test_loader)


Epoch [1/10], Loss: 2.7636
Epoch [2/10], Loss: 2.1165
Epoch [3/10], Loss: 1.6412
Epoch [4/10], Loss: 1.5608
Epoch [5/10], Loss: 1.5506
Epoch [6/10], Loss: 1.5252
Epoch [7/10], Loss: 1.4928
Epoch [8/10], Loss: 1.4576
Epoch [9/10], Loss: 1.4836
Epoch [10/10], Loss: 1.4492
FixMatch Model Evaluation:


/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


              precision    recall  f1-score   support

          H1       0.63      0.97      0.76       881
          H2       0.38      0.18      0.25       467
          H3       0.00      0.00      0.00       164
          H5       0.00      0.00      0.00       163
          H6       0.48      0.78      0.59       148

    accuracy                           0.58      1823
   macro avg       0.30      0.39      0.32      1823
weighted avg       0.44      0.58      0.48      1823

Epoch [1/10], Loss: 3.4397
Epoch [2/10], Loss: 3.4396
Epoch [3/10], Loss: 3.4396
Epoch [4/10], Loss: 3.4396
Epoch [5/10], Loss: 3.4396
Epoch [6/10], Loss: 3.4400
Epoch [7/10], Loss: 3.4397
Epoch [8/10], Loss: 3.4396
Epoch [9/10], Loss: 3.4396
Epoch [10/10], Loss: 3.4395
Mean Teacher Model Evaluation:


/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


              precision    recall  f1-score   support

          H1       0.48      1.00      0.65       881
          H2       0.00      0.00      0.00       467
          H3       0.00      0.00      0.00       164
          H5       0.00      0.00      0.00       163
          H6       0.00      0.00      0.00       148

    accuracy                           0.48      1823
   macro avg       0.10      0.20      0.13      1823
weighted avg       0.23      0.48      0.31      1823

Epoch [1/10], Loss: 1.4153
Epoch [2/10], Loss: 1.4562
Epoch [3/10], Loss: 1.5305
Epoch [4/10], Loss: 1.3876
Epoch [5/10], Loss: 1.4492
Epoch [6/10], Loss: 1.4108
Epoch [7/10], Loss: 1.3506
Epoch [8/10], Loss: 1.3144
Epoch [9/10], Loss: 1.3087
Epoch [10/10], Loss: 1.2907
MixMatch Model Evaluation:
              precision    recall  f1-score   support

          H1       0.62      0.97      0.76       881
          H2       0.47      0.13      0.21       467
          H3       0.45      0.06      0.11       

In [7]:
# Required libraries
import os
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader, Subset

# Set device (GPU/CPU)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Load dataset
data_dir = '/kaggle/input/defungi'  # Change this to your dataset path

# Define basic transformations
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # ResNet input size is 224x224
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),  # Normalization for pre-trained models
])

# Load full dataset
full_dataset = datasets.ImageFolder(root=data_dir, transform=transform)

# Split data into labeled (small fraction) and unlabeled
train_indices, test_indices = train_test_split(list(range(len(full_dataset))), test_size=0.2, stratify=full_dataset.targets)
labeled_indices, unlabeled_indices = train_test_split(train_indices, test_size=0.9, stratify=np.array(full_dataset.targets)[train_indices])

labeled_dataset = Subset(full_dataset, labeled_indices)
unlabeled_dataset = Subset(full_dataset, unlabeled_indices)
test_dataset = Subset(full_dataset, test_indices)

# DataLoaders
batch_size = 32
labeled_loader = DataLoader(labeled_dataset, batch_size=batch_size, shuffle=True)
unlabeled_loader = DataLoader(unlabeled_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

# Load Pretrained Model (ResNet18)
def get_pretrained_model():
    model = models.resnet18(pretrained=True)
    # Modify the final layer to match the number of fungi classes (6 classes in your case)
    num_ftrs = model.fc.in_features
    model.fc = nn.Linear(num_ftrs, 6)  # 6 fungi classes
    return model

# Define training and evaluation function
def train_model(model, optimizer, criterion, labeled_loader, unlabeled_loader, epochs=5):
    model.train()
    for epoch in range(epochs):
        total_loss = 0
        for (labeled_images, labeled_targets), (unlabeled_images, _) in zip(labeled_loader, unlabeled_loader):
            labeled_images, labeled_targets = labeled_images.to(device), labeled_targets.to(device)
            unlabeled_images = unlabeled_images.to(device)

            optimizer.zero_grad()

            # Forward pass on labeled data
            labeled_outputs = model(labeled_images)
            labeled_loss = criterion(labeled_outputs, labeled_targets)

            # Forward pass on unlabeled data
            pseudo_labels = model(unlabeled_images).detach().argmax(dim=1)
            unlabeled_outputs = model(unlabeled_images)
            unlabeled_loss = criterion(unlabeled_outputs, pseudo_labels)

            # Combined loss
            loss = labeled_loss + unlabeled_loss
            loss.backward()
            optimizer.step()

            total_loss += loss.item()

        print(f'Epoch [{epoch+1}/{epochs}], Loss: {total_loss / len(labeled_loader):.4f}')

# FixMatch model (consistency regularization)
class FixMatch(nn.Module):
    def __init__(self, model, tau=0.95):
        super(FixMatch, self).__init__()
        self.model = model
        self.tau = tau

    def forward(self, x_weak, x_strong):
        logits_weak = self.model(x_weak)
        pseudo_labels = torch.softmax(logits_weak, dim=1).argmax(dim=1)
        mask = torch.max(torch.softmax(logits_weak, dim=1), dim=1)[0] >= self.tau
        logits_strong = self.model(x_strong)
        return logits_strong, pseudo_labels, mask

# Mean Teacher model
class MeanTeacher(nn.Module):
    def __init__(self, student, teacher, alpha=0.99):
        super(MeanTeacher, self).__init__()
        self.student = student
        self.teacher = teacher
        self.alpha = alpha

    def update_teacher(self):
        for teacher_param, student_param in zip(self.teacher.parameters(), self.student.parameters()):
            teacher_param.data = self.alpha * teacher_param.data + (1 - self.alpha) * student_param.data

    def forward(self, x):
        return self.student(x), self.teacher(x)

# MixMatch augmentation
def mixmatch(x_labeled, y_labeled, x_unlabeled, model, T=0.5, K=2):
    with torch.no_grad():
        preds_unlabeled = [model(x_unlabeled) for _ in range(K)]
        preds_unlabeled = torch.mean(torch.stack(preds_unlabeled), dim=0)
        pseudo_labels = torch.softmax(preds_unlabeled / T, dim=1)
    all_inputs = torch.cat([x_labeled, x_unlabeled], dim=0)
    all_labels = torch.cat([y_labeled, pseudo_labels.argmax(dim=1)], dim=0)
    lam = np.random.beta(0.75, 0.75)
    index = torch.randperm(all_inputs.size(0))
    mixed_inputs = lam * all_inputs + (1 - lam) * all_inputs[index]
    mixed_labels = lam * all_labels + (1 - lam) * all_labels[index]
    return mixed_inputs, mixed_labels

# Model evaluation
def evaluate_model(model, test_loader):
    model.eval()
    all_preds = []
    all_labels = []
    with torch.no_grad():
        for images, labels in test_loader:
            images = images.to(device)
            outputs = model(images)
            preds = torch.argmax(outputs, dim=1)
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.numpy())
    print(classification_report(all_labels, all_preds, target_names=full_dataset.classes))

# Instantiate the pre-trained model
pretrained_model = get_pretrained_model().to(device)

# Train and evaluate FixMatch model
fixmatch_model = FixMatch(pretrained_model).to(device)
optimizer = optim.Adam(fixmatch_model.parameters(), lr=0.001)
criterion = nn.CrossEntropyLoss()

train_model(fixmatch_model.model, optimizer, criterion, labeled_loader, unlabeled_loader, epochs=32)
print("FixMatch Model Evaluation:")
evaluate_model(fixmatch_model.model, test_loader)

# # Train and evaluate Mean Teacher model
# student_model = get_pretrained_model().to(device)
# teacher_model = get_pretrained_model().to(device)
# mean_teacher_model = MeanTeacher(student_model, teacher_model).to(device)

# train_model(mean_teacher_model.student, optimizer, criterion, labeled_loader, unlabeled_loader, epochs=5)
# print("Mean Teacher Model Evaluation:")
# evaluate_model(mean_teacher_model.student, test_loader)

# Train and evaluate MixMatch model
train_model(pretrained_model, optimizer, criterion, labeled_loader, unlabeled_loader, epochs=32)
print("MixMatch Model Evaluation:")
evaluate_model(pretrained_model, test_loader)


/opt/conda/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=EfficientNet_B2_Weights.IMAGENET1K_V1`. You can also use `weights=EfficientNet_B2_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/efficientnet_b2_rwightman-c35c1473.pth" to /root/.cache/torch/hub/checkpoints/efficientnet_b2_rwightman-c35c1473.pth
100%|██████████| 35.2M/35.2M [00:00<00:00, 192MB/s]


AttributeError: 'EfficientNet' object has no attribute 'fc'

In [12]:
# Full Code with EfficientNet
# Required libraries
import os
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader, Subset

# Set device (GPU/CPU)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Load dataset
data_dir = '/kaggle/input/defungi'  # Change this to your dataset path

# Define basic transformations
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # EfficientNet input size is 224x224
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),  # Normalization for pre-trained models
])

# Load full dataset
full_dataset = datasets.ImageFolder(root=data_dir, transform=transform)

# Split data into labeled (small fraction) and unlabeled
train_indices, test_indices = train_test_split(list(range(len(full_dataset))), test_size=0.2, stratify=full_dataset.targets)
labeled_indices, unlabeled_indices = train_test_split(train_indices, test_size=0.9, stratify=np.array(full_dataset.targets)[train_indices])

labeled_dataset = Subset(full_dataset, labeled_indices)
unlabeled_dataset = Subset(full_dataset, unlabeled_indices)
test_dataset = Subset(full_dataset, test_indices)

# DataLoaders
batch_size = 32
labeled_loader = DataLoader(labeled_dataset, batch_size=batch_size, shuffle=True)
unlabeled_loader = DataLoader(unlabeled_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

# Load Pretrained Model (EfficientNet)
def get_pretrained_model():
    model = models.efficientnet_b2(pretrained=True)  # Use EfficientNet B0
    num_ftrs = model.classifier[1].in_features  # Access the first layer of the classifier
    model.classifier[1] = nn.Linear(num_ftrs, 6)  # 6 fungi classes
    return model

# Define training and evaluation function
def train_model(model, optimizer, criterion, labeled_loader, unlabeled_loader, epochs=5):
    model.train()
    for epoch in range(epochs):
        total_loss = 0
        for (labeled_images, labeled_targets), (unlabeled_images, _) in zip(labeled_loader, unlabeled_loader):
            labeled_images, labeled_targets = labeled_images.to(device), labeled_targets.to(device)
            unlabeled_images = unlabeled_images.to(device)

            optimizer.zero_grad()

            # Forward pass on labeled data
            labeled_outputs = model(labeled_images)
            labeled_loss = criterion(labeled_outputs, labeled_targets)

            # Forward pass on unlabeled data
            pseudo_labels = model(unlabeled_images).detach().argmax(dim=1)
            unlabeled_outputs = model(unlabeled_images)
            unlabeled_loss = criterion(unlabeled_outputs, pseudo_labels)

            # Combined loss
            loss = labeled_loss + unlabeled_loss
            loss.backward()
            optimizer.step()

            total_loss += loss.item()

        print(f'Epoch [{epoch+1}/{epochs}], Loss: {total_loss / len(labeled_loader):.4f}')

# FixMatch model (consistency regularization)
class FixMatch(nn.Module):
    def __init__(self, model, tau=0.95):
        super(FixMatch, self).__init__()
        self.model = model
        self.tau = tau

    def forward(self, x_weak, x_strong):
        logits_weak = self.model(x_weak)
        pseudo_labels = torch.softmax(logits_weak, dim=1).argmax(dim=1)
        mask = torch.max(torch.softmax(logits_weak, dim=1), dim=1)[0] >= self.tau
        logits_strong = self.model(x_strong)
        return logits_strong, pseudo_labels, mask

# Mean Teacher model
class MeanTeacher(nn.Module):
    def __init__(self, student, teacher, alpha=0.99):
        super(MeanTeacher, self).__init__()
        self.student = student
        self.teacher = teacher
        self.alpha = alpha

    def update_teacher(self):
        for teacher_param, student_param in zip(self.teacher.parameters(), self.student.parameters()):
            teacher_param.data = self.alpha * teacher_param.data + (1 - self.alpha) * student_param.data

    def forward(self, x):
        return self.student(x), self.teacher(x)

# MixMatch augmentation
def mixmatch(x_labeled, y_labeled, x_unlabeled, model, T=0.5, K=2):
    with torch.no_grad():
        preds_unlabeled = [model(x_unlabeled) for _ in range(K)]
        preds_unlabeled = torch.mean(torch.stack(preds_unlabeled), dim=0)
        pseudo_labels = torch.softmax(preds_unlabeled / T, dim=1)
    all_inputs = torch.cat([x_labeled, x_unlabeled], dim=0)
    all_labels = torch.cat([y_labeled, pseudo_labels.argmax(dim=1)], dim=0)
    lam = np.random.beta(0.75, 0.75)
    index = torch.randperm(all_inputs.size(0))
    mixed_inputs = lam * all_inputs + (1 - lam) * all_inputs[index]
    mixed_labels = lam * all_labels + (1 - lam) * all_labels[index]
    return mixed_inputs, mixed_labels

# Updated Model evaluation with safeguards
def evaluate_model(model, test_loader):
    model.eval()
    all_preds = []
    all_labels = []
    with torch.no_grad():
        for images, labels in test_loader:
            images = images.to(device)
            outputs = model(images)
            preds = torch.argmax(outputs, dim=1)
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.numpy())

    # Convert to numpy arrays for easier handling
    all_preds = np.array(all_preds)
    all_labels = np.array(all_labels)

    # Get the unique classes from predictions
    unique_classes = np.unique(all_labels)
    print("Unique classes in test set:", unique_classes)

    # Use only the unique classes for the classification report
    print(classification_report(all_labels, all_preds, target_names=[full_dataset.classes[i] for i in unique_classes]))

# Instantiate the pre-trained model
pretrained_model = get_pretrained_model().to(device)

# Train and evaluate FixMatch model
fixmatch_model = FixMatch(pretrained_model).to(device)
optimizer = optim.Adam(fixmatch_model.parameters(), lr=0.001)
criterion = nn.CrossEntropyLoss()

train_model(fixmatch_model.model, optimizer, criterion, labeled_loader, unlabeled_loader, epochs=15)
print("FixMatch Model Evaluation:")
evaluate_model(fixmatch_model.model, test_loader)

# Train and evaluate MixMatch model
mixmatch_model = get_pretrained_model().to(device)  # New model for MixMatch
optimizer = optim.Adam(mixmatch_model.parameters(), lr=0.001)

for epoch in range(5):
    mixmatch_model.train()
    total_loss = 0
    for (labeled_images, labeled_targets), (unlabeled_images, _) in zip(labeled_loader, unlabeled_loader):
        labeled_images, labeled_targets = labeled_images.to(device), labeled_targets.to(device)
        unlabeled_images = unlabeled_images.to(device)

        optimizer.zero_grad()

        # Apply MixMatch
        mixed_inputs, mixed_labels = mixmatch(labeled_images, labeled_targets, unlabeled_images, mixmatch_model)
        outputs = mixmatch_model(mixed_inputs)

        loss = criterion(outputs, mixed_labels.long())
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    print(f'Epoch [{epoch+1}/5], Loss: {total_loss / len(labeled_loader):.4f}')

print("MixMatch Model Evaluation:")
evaluate_model(mixmatch_model, test_loader)


# # Train and evaluate Mean Teacher model
# student_model = get_pretrained_model().to(device)
# teacher_model = get_pretrained_model().to(device)
# mean_teacher_model = MeanTeacher(student_model, teacher_model).to(device)

# train_model(mean_teacher_model.student, optimizer, criterion, labeled_loader, unlabeled_loader, epochs=5)
# print("Mean Teacher Model Evaluation:")
# evaluate_model(mean_teacher_model.student, test_loader)


/opt/conda/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=EfficientNet_B2_Weights.IMAGENET1K_V1`. You can also use `weights=EfficientNet_B2_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Epoch [1/15], Loss: 1.8266
Epoch [2/15], Loss: 1.1422
Epoch [3/15], Loss: 0.8473
Epoch [4/15], Loss: 0.7180
Epoch [5/15], Loss: 0.7266
Epoch [6/15], Loss: 0.5416
Epoch [7/15], Loss: 0.5742
Epoch [8/15], Loss: 0.4859
Epoch [9/15], Loss: 0.4466
Epoch [10/15], Loss: 0.5161
Epoch [11/15], Loss: 0.4262
Epoch [12/15], Loss: 0.4203
Epoch [13/15], Loss: 0.3979
Epoch [14/15], Loss: 0.4740
Epoch [15/15], Loss: 0.3842
FixMatch Model Evaluation:
Unique classes in test set: [0 1 2 3 4]
              precision    recall  f1-score   support

          H1       0.78      0.83      0.81       881
          H2       0.52      0.46      0.49       467
          H3       0.59      0.65      0.62       164
          H5       0.79      0.81      0.80       163
          H6       0.84      0.72      0.78       148

    accuracy                           0.71      1823
   macro avg       0.71      0.70      0.70      1823
weighted avg       0.70      0.71      0.71      1823



/opt/conda/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=EfficientNet_B2_Weights.IMAGENET1K_V1`. You can also use `weights=EfficientNet_B2_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Epoch [1/5], Loss: 0.8416
Epoch [2/5], Loss: 0.6434
Epoch [3/5], Loss: 0.6050
Epoch [4/5], Loss: 0.5533
Epoch [5/5], Loss: 0.5588
MixMatch Model Evaluation:
Unique classes in test set: [0 1 2 3 4]
              precision    recall  f1-score   support

          H1       0.56      1.00      0.72       881
          H2       0.00      0.00      0.00       467
          H3       0.08      0.04      0.05       164
          H5       0.19      0.20      0.19       163
          H6       0.00      0.00      0.00       148

    accuracy                           0.50      1823
   macro avg       0.17      0.25      0.19      1823
weighted avg       0.30      0.50      0.37      1823



/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
